# Appendix 2 | Tract boundaries: how do they work?

In [3]:
from pathlib import Path
from skymap_convert import load_pickle_skymap

package_root = Path.home() / "skymap-to-poly-coords"
raw_skymaps_dir = package_root / "tests" / "data" / "raw_skymaps"
skymap_path = raw_skymaps_dir / "skyMap_lsst_cells_v1_skymaps.pickle"

lsst_skymap = load_pickle_skymap(skymap_path)
lsst_skymap

## Getting multiple tracts by picking a point just above the upper boundary of a tract in ring 0 (first ring after the south pole)

In [10]:
from lsst.geom import Angle, degrees, SpherePoint

In [18]:
# Let's pick tract 2 (avoids RA wrapping)
tract_id = 2
target_tract = lsst_skymap[tract_id]
print(target_tract)

# Then get its inner region
tract_poly = target_tract.getInnerSkyRegion()

# But we should translate these to RA/dec to be human readable (and mathable)
tract_left = tract_poly.getLon().getA().asDegrees()
tract_right = tract_poly.getLon().getB().asDegrees()
print(f"\tTract {tract_id} RA: {tract_left:.2f}, {tract_right:.2f}")

tract_lower = tract_poly.getLat().getA().asDegrees()
tract_upper = tract_poly.getLat().getB().asDegrees()
print(f"\tTract {tract_id} Dec: {tract_lower:.2f}, {tract_upper:.2f}")

# Make a new point that would be just above, at the average of RAs
target_ra = (tract_left + tract_right) / 2
target_dec = tract_upper + 0.01  # Just above the upper bound
print(f"\nTarget RA: {target_ra:.2f}, Target Dec: {target_dec:.2f}")

# And translate that target point to SpherePoint
angle_ra = Angle(target_ra, degrees)
angle_dec = Angle(target_dec, degrees)
target_sphere_point = SpherePoint(angle_ra, angle_dec)
print(f"\tTarget point: {target_sphere_point}")

# Then run this through findTractPatchList
tracts_found = lsst_skymap.findTractPatchList([target_sphere_point])

# And take a look at the tracts we get back
print(f"\nTracts found:")
print("\n".join([str(tract) for tract in tracts_found]))

TractInfo(id=2)
	Tract 2 RA: 18.00, 54.00
	Tract 2 Dec: -89.26, -87.77

Target RA: 36.00, Target Dec: -87.76
	Target point: (36.0000000000, -87.7585950413)

Tracts found:
(TractInfo(id=2, ctrCoord=[0.02100261408048886, 0.015259292328759364, -0.9996629653035124]), (PatchInfo(index=Index2D(x=4, y=9), innerBBox=(minimum=(12000, 27000), maximum=(14999, 29999)), outerBBox=(minimum=(11800, 26800), maximum=(15199, 30199)), cellInnerDimensions=(150, 150), cellBorder=50, numCellsPerPatchInner=22), PatchInfo(index=Index2D(x=5, y=9), innerBBox=(minimum=(15000, 27000), maximum=(17999, 29999)), outerBBox=(minimum=(14800, 26800), maximum=(18199, 30199)), cellInnerDimensions=(150, 150), cellBorder=50, numCellsPerPatchInner=22)))
(TractInfo(id=12, ctrCoord=[0.04795287523125051, 0.01986273127556883, -0.998652088398823]), (PatchInfo(index=Index2D(x=1, y=0), innerBBox=(minimum=(3000, 0), maximum=(5999, 2999)), outerBBox=(minimum=(2800, -200), maximum=(6199, 3199)), cellInnerDimensions=(150, 150), cellBor

- Seems this is ultimately getting a likely tract based on search point's RA
  - Then grabbing adjacent tracts to be safe
- And running tract.contains to see if the point is "in" each of the tracts
- And tract.contains then calls self.getBBox().contains(...)
- And getBBox's docstring says "Get bounding box of tract (as an geom.Box2I)"
  - This seems to be based on the patches
  - So, I don't think this is strictly using the boundaries of the tract
  - Seems like more of a "let's be safe and not sorry" type thing
